In [2]:
import numpy as np
import PIL
import umap
import pandas as pd
import json
import glob
from pytorch_metric_learning import distances, losses, miners, reducers
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import models
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
from tqdm import tqdm
from pytorch_metric_learning import losses, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator
from pytorch_metric_learning.distances import LpDistance,CosineSimilarity
from pytorch_metric_learning.utils.inference import CustomKNN
%matplotlib inline
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
import numpy as np
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
from torchvision import transforms
import torch.optim as optim
import time
import tqdm as tqdm
from torch.autograd import Variable

In [3]:
config = {
    'name':'cub_triplet_loss_epshn_resnet50_sgd_aug',
    'dataset':'CUB_200_2011',
    'random_seed':42,
    'model_architecture':'resnet50',
    'embedding_dim':128,
    'distance':'cosine',
    'image_height':224,
    'image_width':224,
    'train_test_split':0.2,
    'class_split':0.1,
    'batch_size':128,
    'optimizer':'sgd',
    'learning_rate':0.001,
    'num_epochs':100,
    'loss':'NTXentLoss',
    'miner':'epshn',
    'reducer':0,
    'metric':'precision_at_1',
    'model_save_path':'models/cub_triplet_loss_epshn_resnet50_sgd_aug',
    'temperature': 0.1
}

In [4]:
np.random.seed(config['random_seed'])
torch.manual_seed(config['random_seed'])
torch.cuda.manual_seed(config['random_seed'])
torch.backends.cudnn.deterministic = False

In [5]:
# print(models.resnet18())
class ResNetFeatrueExtractor50(nn.Module):
    def __init__(self, pretrained = True):
        super(ResNetFeatrueExtractor50, self).__init__()
        self.model = models.resnet50(pretrained=pretrained)
        self.model.fc = nn.Linear(2048, config['embedding_dim'])

    def forward(self, x):
        x = self.model(x)
        return F.normalize(x)

In [6]:
device = torch.device("cuda")

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# create train and test transforms
transform = transforms.Compose(
    [
        transforms.Resize((config['image_height'], config['image_width'])),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ]
)

batch_size = 128

In [7]:
def tra_transforms(imgsize, RGBmean, RGBstdv):
    return transforms.Compose([transforms.Resize(int(imgsize*1.1)),
                                 transforms.RandomCrop(imgsize),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize(RGBmean, RGBstdv)])

def eva_transforms(imgsize, RGBmean, RGBstdv):
    return transforms.Compose([transforms.Resize(imgsize),
                                 transforms.CenterCrop(imgsize),
                                 transforms.ToTensor(),
                                 transforms.Normalize(RGBmean, RGBstdv)])

In [19]:
train_transform = tra_transforms(224,mean,std)
test_transform = eva_transforms(224,mean,std)

In [9]:
with open('CUB_200_2011/classes.txt','r') as f:
    classes = f.readlines()
classes = [i.replace('\n','') for i in classes]
classes = [i.split(' ')[1] for i in classes]
class_dict = {k:v for k,v in zip(classes,range(200))}

In [10]:
image_paths = []
labels = []
for folder_path,i in class_dict.items():
    folder_images = glob.glob('CUB_200_2011/images/'+'/'+str(folder_path)+'/*')
    image_paths.extend(folder_images)
    labels.extend([i]*len(folder_images))

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_paths, labels, test_size=config['train_test_split'],
                                                    stratify=labels,
                                                    random_state=config['random_seed'])

In [12]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

9430 2358 9430 2358


In [13]:
model = torch.load('models/cub_triplet_loss_epshn_resnet50_sgd_aug_180.pth',map_location='cuda')

In [14]:
model.eval()

ResNetFeatrueExtractor50(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequent

In [16]:
def extract_category(path):
    return path.split('/')[-2]
    
sorted_test_paths = sorted(X_test, key=extract_category)
sorted_train_paths = sorted(X_train, key=extract_category)

In [17]:
rev_class_dict = {v:k for k,v in class_dict.items()}

In [1]:
import fiftyone as fo
import numpy as np
import fiftyone.brain as fob
import fiftyone.zoo as foz
from fiftyone import ViewField as F

In [2]:
name = "CUB-200"
dataset_dir = "CUB_200_2011/images/"
# Create the dataset
dataset = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=fo.types.ImageClassificationDirectoryTree,
    name=name,
)

# View summary info about the dataset
print(dataset)

# Print the first few samples in the dataset
print(dataset.head())

 100% |█████████████| 11788/11788 [1.9s elapsed, 0s remaining, 6.4K samples/s]         
Name:        CUB-200
Media type:  image
Num samples: 11788
Persistent:  False
Tags:        []
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
[<Sample: {
    'id': '6639187385c213e6d9a3e5be',
    'media_type': 'image',
    'filepath': '/home/pranav/embedding_classification/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0001_796111.jpg',
    'tags': [],
    'metadata': None,
    'ground_truth': <Classification: {
        'id': '6639187385c213e6d9a3e5bd',
        'tags': [],
        'label': '001.Black_footed_Albatross',
        'conf

In [3]:
with open('CUB_200_2011/classes.txt','r') as f:
    classes = f.readlines()
classes = [i.replace('\n','') for i in classes]
classes = [i.split(' ')[1] for i in classes]
class_dict = {k:v for k,v in zip(classes,range(200))}

In [6]:
filepaths = dataset.values('filepath')

['/home/pranav/embedding_classification/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0001_796111.jpg',
 '/home/pranav/embedding_classification/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0002_55.jpg',
 '/home/pranav/embedding_classification/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0003_796136.jpg',
 '/home/pranav/embedding_classification/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0005_796090.jpg',
 '/home/pranav/embedding_classification/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0006_796065.jpg',
 '/home/pranav/embedding_classification/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0007_796138.jpg',
 '/home/pranav/embedding_classification/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0008_796083.jpg',
 '/home/pranav/embedding_classification/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatros

## filter classes to last 100, since fiftyone cant show more than 100 classes at once

In [7]:
filepaths = dataset.values('filepath')
with open('fiftyone_filepaths.txt','r') as f:
    filepaths = f.readlines()

In [8]:
filepaths = [i.replace('\n','') for i in filepaths]

In [9]:
pos_view = dataset.filter_labels("ground_truth", F("label").is_in(tuple(classes[100:])))

In [10]:
filtered_paths = (np.array(filepaths)[:, None] == np.array(pos_view.values('filepath'))).argmax(axis=0)

In [11]:
embeddings = np.load('fiftyone_embeddings.npy')
# embeddings = []
# for path in filepaths:
#     img = PIL.Image.open(path.replace('/home/pranav/embedding_classification/',''))
#     if len(img.getbands()) == 1:
#         img = img.convert("RGB")
#     transformed_img = test_transform(img)
#     embedding = model(torch.unsqueeze(transformed_img,dim=0).to(device))
#     embeddings.append(embedding.detach().cpu().numpy()[0])

In [12]:
filtered_embeddings = np.take(embeddings,filtered_paths,axis=0)

In [13]:
# Compute visualization
results = fob.compute_visualization(
    pos_view, embeddings=filtered_embeddings, seed=51, brain_key="resnet50"
)

Generating visualization...


/home/pranav/embedding_classification/fiftyone_env/lib/python3.8/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


UMAP(n_jobs=1, random_state=51, verbose=True)
Mon May  6 23:26:14 2024 Construct fuzzy simplicial set
Mon May  6 23:26:14 2024 Finding Nearest Neighbors
Mon May  6 23:26:14 2024 Building RP forest with 9 trees
Mon May  6 23:26:16 2024 NN descent for 13 iterations
	 1  /  13
	 2  /  13
	 3  /  13
	 4  /  13
	Stopping threshold met -- exiting after 4 iterations
Mon May  6 23:26:21 2024 Finished Nearest Neighbor Search
Mon May  6 23:26:22 2024 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs
Mon May  6 23:26:27 2024 Finished embedding


In [14]:
session = fo.launch_app(pos_view,port=8051)
session.wait()

Notebook sessions cannot wait


In [1]:
import glob
images = glob.glob('CUB_200_2011/images/**/*.jpg')

In [3]:
len(images)

11788